<a href="https://colab.research.google.com/github/Athreya99/IT-Ticket-Chat/blob/main/IT_tickets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch transformers peft bitsandbytes accelerate datasets


In [ ]:
import csv
import json

input_csv = "tickets.csv"
output_jsonl = "tickets.jsonl"

with open(input_csv, newline='', encoding='utf-8') as csvfile, open(output_jsonl, 'w', encoding='utf-8') as jsonlfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # Collect tags into a single list
        tags = [row.get(f"tag_{i}", "") for i in range(1, 9)]
        tags = [t for t in tags if t]  # remove empties

        # Build instruction-response format
        sample = {
            "instruction": "Provide full details for this IT support ticket.",
            "input": row["body"],
            "output": f"Answer: {row.get('answer','')} | Type: {row.get('type','')} | Queue: {row.get('queue','')} | Priority: {row.get('priority','')} | Language: {row.get('language','')} | Tags: {', '.join(tags)}"
        }

        jsonlfile.write(json.dumps(sample, ensure_ascii=False) + "\n")

print(f" JSONL saved to {output_jsonl}")


✅ Done! JSONL saved to tickets.jsonl


In [ ]:
!pip install huggingface_hub
!huggingface-cli login


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `tix` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no

In [ ]:
import torch
torch.cuda.empty_cache()
!pip install -U bitsandbytes

In [ ]:
!pip install -U bitsandbytes transformers accelerate peft datasets

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# -------------------
# 1. Load dataset
# -------------------
dataset = load_dataset("json", data_files="tickets.jsonl")

# -------------------
# 2. Load tokenizer
# -------------------
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# -------------------
# 3. Load model with 4-bit quantization (new API)
# -------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    offload_folder="offload"
)

# Prepare for k-bit training (important!)
model = prepare_model_for_kbit_training(model)

# Enable gradient checkpointing to save VRAM
model.gradient_checkpointing_enable()

# -------------------
# 4. Add LoRA adapters
# -------------------
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# -------------------
# 5. Tokenize data
# -------------------
def format_example(example):
    text = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    tokens = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized = dataset.map(format_example)

print("Sample keys:", tokenized["train"][0].keys())

# -------------------
# 6. Training arguments
# -------------------
training_args = TrainingArguments(
    output_dir="./qlora-ticket-bot",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="steps",
    save_steps=200,
    bf16=True,
    optim="paged_adamw_8bit",
    report_to="none"
)

# -------------------
# 7. Trainer
# -------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"]
)

# -------------------
# 8. Train & save
# -------------------
trainer.train()
model.save_pretrained("./ticket-bot-lora")


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470


Map:   0%|          | 0/28587 [00:00<?, ? examples/s]

Sample keys: dict_keys(['instruction', 'input', 'output', 'input_ids', 'attention_mask', 'labels'])


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,5.977100
20,5.903800
30,5.658500
40,5.752800
50,5.659700
60,5.533500
70,5.605800
80,5.744100
90,5.727100
100,5.735200


In [ ]:
from peft import PeftModel

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)
model = PeftModel.from_pretrained(model, "./ticket-bot-lora")

text = "Who resolved the VPN issue after the Mac update?"
inputs = tokenizer(text, return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]